In [48]:
import pandas as pd
import sys
import os.path
import numpy as np

import bokeh.plotting
from bokeh.plotting import figure, output_server, output_notebook, show, gridplot, reset_output

sys.path.append("/home/daliu/Src/spykshrk_realtime/")

import spykshrk.realtime.binary_record as bin_rec

pd.set_option('float_format', '{:,.1f}'.format)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 120)

output_notebook()

In [49]:
rec_dir = '/opt/data36/daliu/realtime/spykshrk'
file_ids = [1, 2]
file_prefix = 'test_animal'
file_postfix = 'bin_rec'

allrecs = {}

for file_id in file_ids:
    binrec = bin_rec.BinaryRecordsFileReader(file_id=file_id, save_dir=rec_dir, file_prefix=file_prefix, 
                                             file_postfix=file_postfix, filemeta_as_col=False)

    rec_pandas = binrec.convert_pandas()
    
    for rec_id, rec_pd in rec_pandas.items():
        existing_rec_pd = allrecs.setdefault(rec_id, pd.DataFrame())
        allrecs[rec_id] = existing_rec_pd.append(rec_pd, ignore_index=True). \
                sort_values(['current_time']).reset_index(drop=True)


In [53]:
ripple_ntrode5 = allrecs[1][allrecs[1]['ntrode_index']==5]

In [54]:
ripple_ntrode5

In [56]:
s1 = figure()
s1.line(x=ripple_ntrode5['current_time'],
       y=ripple_ntrode5['lfp_data'])

show(s1)